# Kaggle mini course - Machine Learning

1. INTRODUCTION

import pandas as pd
from learntools.core import binder
binder.bind(globals())
from learntools.ml_intermediate.ex1 import *
print("Setup Complete")

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

#first read data
X_full=pd.read_csv('train.csv', index_col='Id')
X_test_full=pd.read_csv('test.csv', index_col='Id')

X_full.head(2)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


In [2]:
#obtain target and predictors
y = X_full.SalePrice # variable that we wanted to predict
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = X_full[features].copy() # we are taking only columns that are in features
X_test = X_test_full[features].copy()

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [3]:
X_train.head(2)

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
Id,,,,,,,
619,11694,2007,1828,0,2,3,9
871,6600,1962,894,0,1,2,5


In [4]:
from sklearn.ensemble import RandomForestRegressor
#defining models

model_1=RandomForestRegressor(n_estimators=50, random_state=0)
#number of trees in the forest (n_estimators)
#number of features considered for splitting at each leaf node (max_features)

Random forest explanation:

Decision trees leave you with a difficult decision. A deep tree with lots of leaves will overfit because each prediction is coming from historical data from only the few houses at its leaf. But a shallow tree with few leaves will perform poorly because it fails to capture as many distinctions in the raw data.

Even today's most sophisticated modeling techniques face this tension between underfitting and overfitting. But, many models have clever ideas that can lead to better performance. We'll look at the random forest as an example.

The random forest uses many trees, and it makes a prediction by averaging the predictions of each component tree. It generally has much better predictive accuracy than a single decision tree and it works well with default parameters. If you keep modeling, you can learn more models with even better performance, but many of those are sensitive to getting the right parameters.

Random state:

It changes. On the other hand if you use random_state=some_number, then you can guarantee that the output of Run 1 will be equal to the output of Run 2, i.e. your split will be always the same. It doesn't matter what the actual random_state number is 42, 0, 21, ... The important thing is that everytime you use 42, you will always get the same output the first time you make the split. This is useful if you want reproducible results, for example in the documentation, so that everybody can consistently see the same numbers when they run the examples. In practice I would say, you should set the random_state to some fixed number while you test stuff, but then remove it in production if you really need a random (and not a fixed) split.

In [5]:
model_2=RandomForestRegressor(n_estimators=100, random_state=0)
model_3=RandomForestRegressor(n_estimators=100, criterion='mae',random_state=0)
#criterion-The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain.

In [6]:
model_4=RandomForestRegressor(n_estimators=200, min_samples_split=20,random_state=0)


Min_samples_split:

The minimum number of samples required to split an internal node:

* If int, then consider min_samples_split as the minimum number.
* If float, then min_samples_split is a fraction and ceil(min_samples_split * n_samples) are the minimum number of samples for each split.

In [7]:
model_5=RandomForestRegressor(n_estimators=100, max_depth=7,random_state=0)
# max_depth - The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.

In [8]:
models=[model_1,model_2,model_3,model_4,model_5]

* To select best model we will define function  that returns mean absolute error (MAE) from validation set
* With the MAE metric, we take the absolute value of each error. This converts each error to a positive number. We then take the average of those absolute errors. This is our measure of model quality.

1. Since models' practical value come from making predictions on new data, we measure performance on data that wasn't used to build the model. The most straightforward way to do this is to exclude some data from the model-building process, and then use those to test the model's accuracy on data it hasn't seen before. This data is called validation data.

In [10]:
from sklearn.metrics import mean_absolute_error

#creating a function

def score_model(model, X_t=X_train, X_v=X_valid, y_t=y_train, y_v=y_valid):
    model.fit(X_t,y_t)
    preds=model.predict(X_v)
    return mean_absolute_error(y_v,preds)

# we are taking model, x and y train and valied data, then we are doing fit 
# with train data, then prediction vwith X valid data
# returning MAE between predicted data and valisation data

In [11]:
for i in range(0, len(models)):
    mae=score_model(models[i])
    print("Model %d MAE: %d" % (i+1, mae))

Model 1 MAE: 24015
Model 2 MAE: 23740
Model 3 MAE: 23528
Model 4 MAE: 23996
Model 5 MAE: 23706


In [14]:
# best model is model 3!


In [16]:
model_3.fit(X,y)
preds_test=model_3.predict(X_test)

#save preditions in format used for scoring

output=pd.DataFrame({'Id': X_test.index, 'SalePrice':preds_test})
output.to_csv('Prediction.csv', index=False)